<a href="https://colab.research.google.com/github/yvrjsharma/MLApps_with_Gradio/blob/main/Pix2PixZero_Demo_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
#pip install Pillow==9.0.0 --q
!pip install --ignore-installed Pillow==9.0.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached Pillow-9.0.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.3 MB)


In [2]:
!pip install transformers accelerate diffusers --q

In [3]:
! pip install salesforce-lavis --q

In [4]:
! git clone https://github.com/pix2pixzero/pix2pix-zero.git

fatal: destination path 'pix2pix-zero' already exists and is not an empty directory.


In [5]:
! pip install git+https://github.com/pix2pixzero/pix2pix-zero.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/pix2pixzero/pix2pix-zero.git to /tmp/pip-req-build-zmosz2di
  Running command git clone --filter=blob:none --quiet https://github.com/pix2pixzero/pix2pix-zero.git /tmp/pip-req-build-zmosz2di
  Resolved https://github.com/pix2pixzero/pix2pix-zero.git to commit 25b885767bf81fc8bd9e7aa7a2a095ecbebab9db
ERROR: git+https://github.com/pix2pixzero/pix2pix-zero.git does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [6]:
%cd pix2pix-zero

/content/pix2pix-zero


In [7]:
!pwd

/content/pix2pix-zero


In [9]:
# First, running the inversion command to obtain the input noise that will  reconstruct the image. 
# It will save the inversion as output/test_cat/inversion/cat_1.pt 
# and BLIP-generated prompt as output/test_cat/prompt/cat_1.txt - a painting of a cat sitting on top of a ball
! python src/inversion.py  \
        --input_image "assets/test_images/cats/cat_1.png" \
        --results_folder "output/test_cat"

2023-02-11 16:42:59.443740: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-11 16:42:59.645586: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-11 16:43:00.474182: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.8/dist-packages/cv2/../../lib64:/usr/lib64-nvidia
2023-02-11 16:

In [31]:
# Next, perform image editing with the editing direction as shown. 
# The command will save the edited image as output/test_cat/edit/cat_1.png
! python src/edit_real.py \
    --inversion "output/test_cat/inversion/cat_1.pt" \
    --prompt "output/test_cat/prompt/cat_1.txt" \
    --task_name "cat2dog" \
    --results_folder "output/test_cat/" 

2023-02-11 19:13:52.857177: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-11 19:13:53.012536: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-11 19:13:53.784729: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-11 19:13:53.784824: W tensorflow/compiler/xla/stream_executor

In [74]:
import subprocess
from PIL import Image

# First, running the inversion command to obtain the input noise that will  reconstruct the image. 
# It will save the inversion as output/test_cat/inversion/cat_1.pt 
# and BLIP-generated prompt as output/test_cat/prompt/cat_1.txt - a painting of a cat sitting on top of a ball
def inversion(image_in, progress=gr.Progress(track_tqdm=True)):
  progress(0, desc="Starting...")
  #img = Image.open(image_in)
  image_in.save("assets/test_images/cats/input_image.png")
  # Run the script.py file
  subprocess.run(["python", "src/inversion.py", "--input_image", "input_image.png", "--results_folder", "output/test_cat"])
  # Open the text file with blip caption
  with open("output/test_cat/prompt/input_image.txt", "r") as file:
      # Read the file line by line
      prompt = file.read()
  return "output/test_cat/inversion/input_image.pt", prompt


# Next, perform image editing with the editing direction as shown. 
# The command will save the edited image as output/test_cat/edit/cat_1.png
def image_edit(task_name, progress=gr.Progress(track_tqdm=True)):
  progress(0, desc="Starting...")
  # Run the script.py file
  subprocess.run(["python", "src/edit_real.py", "--inversion", "output/test_cat/inversion/input_image.pt", 
                  "--prompt", "output/test_cat/prompt/input_image.txt", "--task_name", task_name,
                  "--results_folder", "output/test_cat/"])
  return "output/test_cat/edit/input_image.png"
  

#Similarly, we can edit the synthetic images generated by Stable Diffusion with the following command.
def synthetic_image_edit(prompt, task_name, progress=gr.Progress(track_tqdm=True)):
  progress(0, desc="Starting...")
  # Run the script file
  subprocess.run(["python", "src/edit_synthetic.py", "--prompt_str", prompt, 
                  "--task", task_name, "--results_folder", "output/synth_editing"])
  return "output/synth_editing/reconstruction.png", "output/synth_editing/edit.png"
  


#def initial_radio_config(gr.Radio()):
#    if gr.Radio() == "Edit Synthetic (SD) Images":
#        return gr.update(visible=True), gr.update(visible=True)
    

#def button_visiblity():
#    return gr.update(visible=False)

def set_visible_true():
    return gr.update(visible=True)

def set_visible_False():
    return gr.update(visible=False)

In [75]:
with gr.Blocks() as demo:
  gr.HTML("""<div style="text-align: center; max-width: 700px; margin: 0 auto;">
                  <div
              style="
                  display: inline-flex;
                  align-items: center;
                  gap: 0.8rem;
                  font-size: 1.75rem;
              "
              >
              <h1 style="font-weight: 900; margin-bottom: 7px; margin-top: 5px;">
                  Pix2Pix - Zero-shot Image-to-Image Translation
              </h1>
              </div>
              <p style="margin-bottom: 10px; font-size: 94%">
              This is an unofficial demo for <a href="https://pix2pixzero.github.io/" target="_blank">Pix2PixZero</a>. 
              Please visit their website and <a href="https://github.com/pix2pixzero/pix2pix-zero" target="_blank">github repo</a> for more details.
              </p></div>""")
  #task_initial_radio  = gr.Radio(choices = ["Image Translation", "Edit Synthetic (SD) Images",], type="value", visible=True, label="Select a task that you want to perform using Pix2Pix-Zero") 
  with gr.Row():
    image_in = gr.Image(type="pil", label="Upload the image of a cat or a dog you want to translate")
    with gr.Column():
      btn_inversion = gr.Button("Get input noise",visible=False )
      with gr.Row():
        blip_prompt = gr.Textbox(visible=False)
        inversion_file = gr.File(visible=False)
        #task_name = gr.Textbox()
  with gr.Row():
    image_out = gr.Image(visible=False, label="Trnaslated Image output")
    with gr.Column():
      task_name_radio  = gr.Radio(choices = ["cat2dog", "dog2cat",], type="value", visible=False, label="Select a task that you want to accomplish") #, value="cat2dog"),
      btn_imageedit = gr.Button(value="Edit the image!",visible=False)
      prompt_synth = gr.Textbox(visible=False)
      btn_synth_image = gr.Button(value="Generate & Translate SD image",visible=False)
  with gr.Row():
    image_synth = gr.Image(visible=False, label="Synthetic image generated by Stable Diffusion on the fly")
    image_synth_translated = gr.Image(visible=False, label="Translated synthetic image")

    btn_inversion.click(inversion,[image_in],[inversion_file, blip_prompt])
    btn_inversion.click(set_visible_true, [], task_name_radio)  #inversion_file, blip_prompt,
    btn_inversion.click(set_visible_False, [], btn_inversion)

    #task_initial_radio.change(set_visible_true, [], btn_imageedit)
    
    task_name_radio.change(set_visible_true, [], btn_imageedit)
    task_name_radio.change(set_visible_true, [], image_out)

    btn_imageedit.click(image_edit,[task_name_radio],[image_out])
    btn_imageedit.click(set_visible_False, [], btn_imageedit)
    btn_imageedit.click(set_visible_true, [], prompt_synth)
    btn_imageedit.click(set_visible_true, [], btn_synth_image)
    btn_imageedit.click(set_visible_true, [], image_synth)
    btn_imageedit.click(set_visible_true, [], image_synth_translated)
    btn_synth_image.click(synthetic_image_edit,[prompt_synth, task_name_radio],[image_synth, image_synth_translated])

    image_in.clear(set_visible_true, [], btn_inversion)
    image_in.change(set_visible_true, [], btn_inversion)
    image_in.change(set_visible_true, [], blip_prompt)
    image_in.change(set_visible_true, [], inversion_file)

demo.queue()
demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://f23e6ea4-0ec1-4ebd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://f23e6ea4-0ec1-4ebd.gradio.live


## Inversion - my own
Testing for other images then cat and dog
- Results only available tasks are - cat2dog and dog2cat

In [11]:
#inversion
! python src/inversion.py  \
        --input_image "assets/test_images/cats/shiba1.jpg" \
        --results_folder "output/test_cat"

2023-02-11 17:24:19.079208: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-11 17:24:19.281621: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-11 17:24:20.106808: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.8/dist-packages/cv2/../../lib64:/usr/lib64-nvidia
2023-02-11 17:

In [12]:
#image editing with directions
# blip desc - a brown and white dog standing in front of a white background
! python src/edit_real.py \
    --inversion "output/test_cat/inversion/shiba1.pt" \
    --prompt "output/test_cat/prompt/cat_1.txt" \
    --task_name "dog2cat" \
    --results_folder "output/test_cat/" 

2023-02-11 17:26:21.216701: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-11 17:26:21.372151: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-11 17:26:22.152430: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-11 17:26:22.152521: W tensorflow/compiler/xla/stream_executor

In [ ]:
!pip install gradio
import gradio as gr

#Editing images on the fly while generating through SD

In [14]:
#
! python src/edit_synthetic.py \
    --results_folder "output/synth_editing" \
    --prompt_str "a high resolution painting of a cat in the style of van gough" \
    --task "cat2dog"

2023-02-11 17:37:22.871869: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-11 17:37:23.031386: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
Fetching 16 files: 100% 16/16 [00:00<00:00, 195652.66it/s]
100% 50/50 [00:09<00:00,  5.02it/s]
100% 50/50 [00:19<00:00,  2.50it/s]


## Custom edit instructions

In [15]:
! python src/make_edit_direction.py \
    --file_source_sentences sentences/apple.txt \
    --file_target_sentences sentences/orange.txt \
    --output_folder assets/embeddings_sd_1.4

2023-02-11 17:48:15.759994: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-11 17:48:15.914254: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-11 17:48:16.666646: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-11 17:48:16.666751: W tensorflow/compiler/xla/stream_executor

## Trying something else now  -

In [ ]:
# First, running the inversion command to obtain the input noise that will  reconstruct the image. 
# It will save the inversion as output/test_cat/inversion/cat_1.pt 
# and BLIP-generated prompt as output/test_cat/prompt/cat_1.txt - a painting of a cat sitting on top of a ball
! python src/inversion.py  \
        --input_image "assets/test_images/cats/cat_1.png" \
        --results_folder "output/test_cat"

In [18]:
# Next, perform image editing with the editing direction as shown. 
# The command will save the edited image as output/test_cat/edit/cat_1.png
! python src/edit_real.py \
    --inversion "output/test_cat/inversion/cat_1.pt" \
    --prompt "output/test_cat/prompt/cat_2.txt" \
    --results_folder "output/test_cat/" 

2023-02-11 18:08:03.385624: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-11 18:08:03.542149: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-11 18:08:04.305956: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-11 18:08:04.306050: W tensorflow/compiler/xla/stream_executor

In [19]:
#
! python src/edit_synthetic.py \
    --results_folder "output/synth_editing" \
    --prompt_str "a painting of a tiger sitting on top of a ball" \
    

2023-02-11 18:15:20.524434: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-11 18:15:20.679830: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-11 18:15:21.442744: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-11 18:15:21.442836: W tensorflow/compiler/xla/stream_executor